# Quick reference for the Lambda Notebook

## 1. Metalanguage

Classes in `lamb.meta` deal with metalanguage elements.  The metalanguage is the primary thing you write in `%%lamb` cells, as well as with `%te`.

### Metalanguage operators

* `~`: $\neg$ (negation at type `t`)
* `&`: $\wedge$ (conjunction at type `t`)
* `|`: $\vee$ (disjunction at type `t`)
* `>>`, `==>`: $\rightarrow$
* `==`, `<=>`, `%`: equality at various types.  For `t`, this will produce $\leftrightarrow$.
* `=/=`, `^`: non-equality at various types.
* `<<`: set membership / $\in$
* numeric operators are just the python ones (`<`, `<=`, `>`, `>=`, `-`, `+`, `*`, `/`, `**`)

### Metalanguage binding operators

* `Lambda x_X: `$\phi$ (build a lambda term)
  - Also works: `L x:`$\phi$, `λ x:`$\phi$
* `Exists x_X: `$\phi_t$
* `Forall x_X: `$\phi_t$
* `Iota x_X: `$\phi_t$ (n.b.: no semantics is implemented for the uniqueness constraint)
* `Set x_X: `$\phi_t$ (build a set based on some conditions)

### Other metalanguage elements

* tuples: write `(a1,...an)`
* sets by list: write `{a1,...an}`
* terms:
  - variables: lowercase followed by underscore and type, e.g. `x_e`
  - constants: uppercase, e.g. `P_<e,t>`
* To add partiality conditions use the call `Partial` in either the metalanguage or in python, e.g. `L x_e : Partial(Body(x), Condition(x))`.  Effective use of partiality conditions requires the correct composition system and some care; see [Partiality documentation.ipynb](http://localhost:8888/notebooks/documentation/Partiality%20documentation.ipynb) for more details.

Built-in constants `True_t` and `False_t` make up type `t`

### magics

* the `%te` line magic is basically the same as calling the `te` function on a string, and parses a meta-language expression


### Working with metalanguage elements in python

The function `te` parses a string into a metalanguage object.
* advanced: it isn't guaranteed to finish cleaning up type inference if there is polymorphism impacting free variables/constants.  If you run into problems with this, a safe strategy is to call `te(s, let=True)`.
* advanced: `te` can also take an optional assignment.

For any metalanguage objects `m` and `m2`:
* `m.type` gives m's type (as a type object)
* `m.derivation` gives derivational history, if any
* `m.reduce_all()` reduces any lambda expressions.
* `m.simplify_all()` simplifies certain things.  (Not everything that could be simplified is implemented.)
* `m(m2)` attempts to build a function-argument expression from `m` as function and `m2` as argument.  Type checking/inference is enforced, but no reduction happens.
  - `m(m2).reduce_all()` does reduction as well.  This is what you have probably been trained to expect from programming languages.
* `m == m2` checks syntactic equality.
* For most other python operators, `m op m2` produces a complex TypedExpr based on the behavior in the metalanguage parser.  Certain special cases that aren't python operators don't work in python.  In some special cases other things are overriden in python (e.g. `*` for functions does function composition).
* `m.copy()` should work as expected for any metalanguage object.  It usually returns a shallow copy.
* `m.under_assignment(a)` uses `a` as a dict mapping variable names to metalanguage values operating on any free terms in `a`.
* `m.free_variables()` returns a python set containing any variables free in `m`.
* `m.term()` returns true iff `m` is a term.  (Safer than checking using `isinstance`).
* advanced: `m.type_env()` shows the complete type inference environment in `m`.
* advanced: `m[0]` gives the operator, which may be a string, and `m[1]` etc give arguments.  Generally there are 0, 1 or 2 arguments (terms, unary combinations, binary operators).  Exceptions: tuples, listed sets.

## 2. Types

Classes in `lamb.types` deal with types and tyep inference.

The function `meta.tp` calls the type parser on some type string.

* atomic types by default: `e`, `t`, `n`
* polymorphic types: `X`, `Y`, `Z`, `X1`, etc.
* functions types: `<X,Y>` for a function from arbitrary type `X` to arbitrary type `Y`
* tuples: `(X1,X2,...)` for types `X1` etc
* sets: `{X}` for a set of elements of type `X`
      
### Working with types in python

To add a new atomic type in python (example using type `s`):

      meta.get_type_system().add_atomic(types.BasicType("s"))

* the type system can be found at `meta.get_type_system()`.
* you can use `==`, but it will check exact equality.
* for type equality in the general case use `meta.get_type_system().unify(a,b)`
  - shortcut: `meta.ts_unify(a,b)`
  - returns `None` if there is no principle type.
  - otherwise, returns the principle type.  May not be equal to `a` or to `b`.
  - if `a` and `b` are equal, it will return that value.
  - if `a` and `b` are not polymorphic, it reduces to equality checking.
  - to check compatibility, there is a convenience function `meta.ts_compatible(a,b)` that returns `True` or `False`.

## 3. Composition operations and systems

Classes in `lamb.lang` represent object language elements and the composition systems that put them together.

### In python cells

* the `*` operator composes two Composables
* for any (unambiguous) lexical item, `item.content` gets the metalanguage content of the item
* for composition results and ambiguous lexical items (call it `r`):
   - `r` is indexed, `len(r)` tells you how many contents it has, and `r[2].content` extracts some metalanguage content for element 2
   - `r.trace()` shows a step-by-step derivation for `r`
   - `r.tree()` shows the tree derivation for `r`
* the current composition system can be gotten and set with `lang.get_system()` and `lang.set_system(s)` respectively.
* built-in systems: 
   - `lang.td_system` (default: type-driven bottom-up composition)
   - `lang.hk3_system` (type-driven composition in trees, based on Heim & Kratzer ch 3)
   - `lang.td_presup` (type-driven bottom-up composition with presuppositions via partiality)

### magics

* the `%%lamb` cell magic parses assignments to variables or items
* arguments:
  - `reset`, clear the lexicon
  - `ambiguity`, allow ambiguous items with no annotation in the current cell
  - `all`, show the entire lexicon
* you can supply `reset` and `all` to the %lambctl line magic
* lines of a `%%lamb` magic can look like:
  - `||item|| = blah` (where `item` is an item name, possibly indexed, and `blah` is a metalanguage expression
  - `x = blah` (where `x` is a variable name)
  - advanced: can write `=<t>` for some transform `t` (see the Hamblin semantics notebook)
  - `||item[*]|| = blah` adds a new sense to an existing item

## Various settings and module-level things

* `meta.suppress_bound_var_types`: set to False to show types for bound variables (default: True)
* `meta.suppress_constant_predicate_type`: set to False to show predicate types (default: True)
* `meta.suppress_constant_type`: set to True to suppress all constant types (default: False)
* `lang.bracket_setting`:
  - `lang.BRACKET_BARS`: use just `||`.
  - `lang.BRACKET_FANCY`: use mathjax "`[\![\\mathbf{\\text{" + s + "}}]\!]`".  Best looking, but slower.  (default)
  - `lang.BRACKET_UNI`: use unicode ⟦ and ⟧.
* `utils.use_nltk`: use full NLTK.  **experimental**, currently kind of buggy.
  - utils.get_tree_class() returns the class constructor for whatever tree class is currently in use.
